In [22]:
import pandas as pd
import numpy as np
import math
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA                 # for dimensionality reduction using PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
# from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt 
import seaborn as sns            # visualization tool
import matplotlib.cm as cm       # for colour mapping to use for the pca plots

In [23]:
def zero_crossing(temp,threshold):
    zc = 0
    for i in range(len(temp)-1):
        if((temp[i]*temp[i+1]) < 0 and abs(temp[i]-temp[i+1]) >= threshold):
            zc += 1
    return zc

In [24]:
def process_one_csv(file_name,k):
    data = pd.read_csv('/home/amaya/emp/Realtime-Sign-Language-Translation-to-Speech-DNN/all/User'+str(k)+'/'+file_name)
    global classes
    
    class_ = file_name[0:len(file_name)-14]
    
    # print(data.head(2))

    columns = list(data.columns)[1:]
    # features = ['MAV','RMS','VAR','SSI','MAX','MIN','ZC']

    vector = []
    # global out_columns
    
    for item in columns:
        temp = list(data[item])
        # calculating MAV ***
        # out_columns.append(item+features[0])
        abs_val = list(map(abs,temp))
        vector.append(np.mean(abs_val))

        # calculating RMS ***
        # out_columns.append(item+features[1])
        vector.append(np.sqrt(np.mean(np.array(temp)**2)))

        # calculate variance ***
        # out_columns.append(item+features[2])
        x_mean = np.mean(temp)
        dif = temp - x_mean
        vector.append(np.mean(np.array(dif)**2))

        # calculating ssi ***
        # out_columns.append(item+features[3])
        vector.append(np.sum(np.array(temp)**2))

        # calculating max ***
        # out_columns.append(item+features[4])
        vector.append(max(temp))

        # calculating min ***
        # out_columns.append(item+features[5])
        vector.append(min(temp))
        
        # calculating zero crossing
        if((item[0] == 'E') or (item[0] == 'G')):
            vector.append(zero_crossing(temp,1))
        elif(item[0] == 'A'):
            vector.append(zero_crossing(temp,0.15))
        
    if class_ in classes:
        vector.append(classes.index(class_))
    else:
        classes.append(class_)
        vector.append(classes.index(class_))
        
    return vector

In [25]:
out_columns = ['EMG0LMAV', 'EMG0LRMS', 'EMG0LVAR', 'EMG0LSSI', 'EMG0LMAX', 'EMG0LMIN','EMG0ZC', 'EMG1LMAV', 'EMG1LRMS', 'EMG1LVAR', 'EMG1LSSI', 'EMG1LMAX', 'EMG1LMIN','EMG1ZC', 'EMG2LMAV', 'EMG2LRMS', 'EMG2LVAR', 'EMG2LSSI', 'EMG2LMAX', 'EMG2LMIN', 'EMG2ZC','EMG3LMAV', 'EMG3LRMS', 'EMG3LVAR', 'EMG3LSSI', 'EMG3LMAX', 'EMG3LMIN', 'EMG3ZC','EMG4LMAV', 'EMG4LRMS', 'EMG4LVAR', 'EMG4LSSI', 'EMG4LMAX', 'EMG4LMIN', 'EMG4ZC','EMG5LMAV', 'EMG5LRMS', 'EMG5LVAR', 'EMG5LSSI', 'EMG5LMAX', 'EMG5LMIN', 'EMG5ZC','EMG6LMAV','EMG6LRMS', 'EMG6LVAR', 'EMG6LSSI', 'EMG6LMAX', 'EMG6LMIN', 'EMG6ZC','EMG7LMAV', 'EMG7LRMS', 'EMG7LVAR', 'EMG7LSSI', 'EMG7LMAX', 'EMG7LMIN', 'EMG7ZC','AXLMAV', 'AXLRMS', 'AXLVAR', 'AXLSSI', 'AXLMAX', 'AXLMIN', 'AXLZC','AYLMAV', 'AYLRMS', 'AYLVAR', 'AYLSSI', 'AYLMAX', 'AYLMIN', 'AYLZC','AZLMAV', 'AZLRMS', 'AZLVAR', 'AZLSSI', 'AZLMAX', 'AZLMIN', 'AZLZC','GXLMAV', 'GXLRMS', 'GXLVAR', 'GXLSSI', 'GXLMAX', 'GXLMIN', 'GXLZC','GYLMAV', 'GYLRMS', 'GYLVAR', 'GYLSSI', 'GYLMAX', 'GYLMIN', 'GYLZC','GZLMAV', 'GZLRMS', 'GZLVAR', 'GZLSSI', 'GZLMAX', 'GZLMIN', 'GZLZC','ORLMAV', 'ORLRMS', 'ORLVAR', 'ORLSSI', 'ORLMAX', 'ORLMIN', 'OPLMAV', 'OPLRMS', 'OPLVAR', 'OPLSSI', 'OPLMAX', 'OPLMIN', 'OYLMAV', 'OYLRMS', 'OYLVAR', 'OYLSSI', 'OYLMAX', 'OYLMIN', 'EMG0RMAV', 'EMG0RRMS', 'EMG0RVAR', 'EMG0RSSI', 'EMG0RMAX', 'EMG0RMIN', 'EMG0RZC', 'EMG1RMAV', 'EMG1RRMS', 'EMG1RVAR', 'EMG1RSSI', 'EMG1RMAX', 'EMG1RMIN','EMG1RZC', 'EMG2RMAV', 'EMG2RRMS', 'EMG2RVAR', 'EMG2RSSI', 'EMG2RMAX', 'EMG2RMIN','EMG2RZC', 'EMG3RMAV', 'EMG3RRMS', 'EMG3RVAR', 'EMG3RSSI', 'EMG3RMAX', 'EMG3RMIN','EMG3RZC', 'EMG4RMAV', 'EMG4RRMS', 'EMG4RVAR', 'EMG4RSSI', 'EMG4RMAX', 'EMG4RMIN','EMG4RZC', 'EMG5RMAV', 'EMG5RRMS', 'EMG5RVAR', 'EMG5RSSI', 'EMG5RMAX', 'EMG5RMIN','EMG5RZC', 'EMG6RMAV', 'EMG6RRMS', 'EMG6RVAR', 'EMG6RSSI', 'EMG6RMAX', 'EMG6RMIN','EMG6RZC', 'EMG7RMAV', 'EMG7RRMS', 'EMG7RVAR', 'EMG7RSSI', 'EMG7RMAX', 'EMG7RMIN','EMG7RZC', 'AXRMAV', 'AXRRMS', 'AXRVAR', 'AXRSSI', 'AXRMAX', 'AXRMIN','AXRZC', 'AYRMAV', 'AYRRMS', 'AYRVAR', 'AYRSSI', 'AYRMAX', 'AYRMIN','AYRZC', 'AZRMAV', 'AZRRMS', 'AZRVAR', 'AZRSSI', 'AZRMAX', 'AZRMIN','AZRZC', 'GXRMAV', 'GXRRMS', 'GXRVAR', 'GXRSSI', 'GXRMAX', 'GXRMIN','GXRZC', 'GYRMAV', 'GYRRMS', 'GYRVAR', 'GYRSSI', 'GYRMAX', 'GYRMIN','GYRZC', 'GZRMAV', 'GZRRMS', 'GZRVAR', 'GZRSSI', 'GZRMAX', 'GZRMIN','GZRZC', 'ORRMAV', 'ORRRMS', 'ORRVAR', 'ORRSSI', 'ORRMAX', 'ORRMIN', 'OPRMAV', 'OPRRMS', 'OPRVAR', 'OPRSSI', 'OPRMAX', 'OPRMIN', 'OYRMAV', 'OYRRMS', 'OYRVAR', 'OYRSSI', 'OYRMAX', 'OYRMIN','CLASS']
# 205 columns (34*6 + 1)
output = []
classes=[]
# print(len(out_columns))

# process_one_csv(files[0])
for k in range(1,11):
    if(k!=5):
        files = os.listdir('/home/amaya/emp/Realtime-Sign-Language-Translation-to-Speech-DNN/all/User'+str(k)+'/')
    else:
        continue
    for file in files:
        if(file[-1] == 'v'):      # to avoid _DS_STORE files and anyother hiiden files
            output.append(process_one_csv(file,k))
        
processed = pd.DataFrame(output,columns = out_columns)
processed.shape

(550, 233)

In [26]:
# feature scaling of data
scaler = preprocessing.MinMaxScaler()   # since the data set is not gaussian
scaled_df = scaler.fit_transform(processed[out_columns[102:len(out_columns)-1]])
X = pd.DataFrame(scaled_df, columns = out_columns[102:len(out_columns)-1])
y = processed['CLASS']

In [27]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.20)

In [33]:
# Random Forest

model = RandomForestClassifier(n_estimators = 100)
sel = SelectFromModel(model)
#model.fit(X_train,y_train)
sel.fit(X_train,y_train)
selected_features = X_test.columns[sel.get_support()]
print(selected_features)

52


In [29]:
#print(selected_features)

In [30]:
rf_predictions = model.predict(X_test)
rf_probs = model.predict_proba(X_test)[:, 1]

In [31]:
print(classification_report(y_test,rf_predictions))

              precision    recall  f1-score   support

           0       1.00      0.89      0.94         9
           1       1.00      1.00      1.00         6
           2       1.00      1.00      1.00        10
           3       0.67      1.00      0.80         4
           4       1.00      0.80      0.89         5
           5       1.00      1.00      1.00         7
           6       0.90      1.00      0.95         9
           7       1.00      1.00      1.00         4
           8       1.00      1.00      1.00         7
           9       0.83      1.00      0.91         5
          10       1.00      1.00      1.00         3
          11       1.00      0.50      0.67         4
          12       0.88      1.00      0.93         7
          13       0.67      0.80      0.73         5
          14       1.00      1.00      1.00         2
          15       1.00      0.57      0.73         7
          16       1.00      1.00      1.00         4
          17       0.83    

In [32]:
f1 = pd.DataFrame({'feature': list(X_train.columns),
                   'importance': model.feature_importances_}).\
                   sort_values('importance', ascending = False)
f1

,feature,importance
4,OPLVAR,0.038148
10,OYLVAR,0.032988
7,OPLMIN,0.031876
114,ORRVAR,0.020561
73,AXRSSI,0.017216
79,AYRVAR,0.015402
84,AZRMAV,0.015262
71,AXRRMS,0.015102
91,GXRMAV,0.014393
123,OPRMIN,0.014212


In [10]:
# print(rf_probs)

[0.   0.   0.03 0.02 0.02 0.   0.01 0.02 0.   0.04 0.   0.   0.48 0.03
 0.   0.   0.01 0.12 0.09 0.05 0.05 0.   0.09 0.   0.02 0.   0.   0.
 0.   0.   0.   0.01 0.01 0.02 0.09 0.   0.04 0.   0.   0.   0.06 0.02
 0.   0.   0.07 0.   0.   0.12 0.01 0.   0.01 0.   0.11 0.   0.02 0.04
 0.01 0.   0.13 0.   0.   0.51 0.   0.01 0.   0.   0.02 0.   0.17 0.08
 0.   0.   0.   0.   0.02 0.05 0.04 0.01 0.01 0.   0.   0.   0.01 0.68
 0.04 0.03 0.06 0.02 0.07 0.   0.03 0.08 0.   0.   0.   0.07 0.09 0.
 0.06 0.32 0.   0.01 0.02 0.   0.   0.   0.   0.13 0.   0.03]
